# Classes & Methods — Tic-tac-toe (Object-Oriented)

This lesson is based directly on the class-based Tic-tac-toe implementation in `2025-08-18-tictactoe-code.ipynb`. All code cells below are taken from that notebook (unchanged or only lightly formatted) so you can study how `class`, `__init__`, methods, and object collaboration work in a real example.

## What you'll learn

- How to define classes with `__init__` and instance variables
- How to add methods that operate on instance state
- How classes collaborate (composition)
- How to test classes with short demos

In [ ]:
# Player class (from the source notebook)
class Player:
    def __init__(self, name, symbol, is_ai=False):
        self.name = name
        self.symbol = symbol
        self.is_ai = is_ai
        self.wins = 0
        self.moves = []
    
    def add_win(self):
        self.wins += 1
        print(f"🏆 {self.name} now has {self.wins} wins!")
    
    def get_move(self, board):
        if not self.is_ai:
            return None
        # AI Logic (delegates to helper)
        return self._get_best_move(board)
    
    def _get_best_move(self, board):
        # First, try to win
        for i in range(9):
            if board.grid[i] == " ":
                board.grid[i] = self.symbol
                if board.check_winner(self.symbol):
                    board.grid[i] = " "  # Reset the move
                    return i + 1
                board.grid[i] = " "  # Reset the move
        
        # Then, block opponent's win
        opponent_symbol = "O" if self.symbol == "X" else "X"
        for i in range(9):
            if board.grid[i] == " ":
                board.grid[i] = opponent_symbol
                if board.check_winner(opponent_symbol):
                    board.grid[i] = " "  # Reset the move
                    return i + 1
                board.grid[i] = " "  # Reset the move today
        
        # Try to get center
        if board.grid[4] == " ":
            return 5
        
        # Try corners
        corners = [0, 2, 6, 8]
        import random
        random.shuffle(corners)  # Add randomness to corner selection
        for corner in corners:
            if board.grid[corner] == " ":
                return corner + 1
        
        # Take any available space
        for i in range(9):
            if board.grid[i] == " ":
                return i + 1

# Quick test of Player creation
p1 = Player('Alice', 'X')
p2 = Player('Bob', 'O')
print(p1.name, p1.symbol, p1.is_ai)
print(p2.name, p2.symbol, p2.is_ai)

## Board class — encapsulates grid and rules

Below is the `Board` implementation copied from the notebook. It contains display helpers, move validation, undo, and win detection.

In [ ]:
from colorama import Fore, Style

class Board:
    def __init__(self):
        self.grid = [" "] * 9
        self.move_history = []

    def display(self):
        print("\n")
        colors = {" ": Style.RESET_ALL, "X": Fore.RED, "O": Fore.BLUE}
        for i in range(0, 9, 3):
            print(" " + colors[self.grid[i]] + self.grid[i] + Style.RESET_ALL + " | " + 
                  colors[self.grid[i+1]] + self.grid[i+1] + Style.RESET_ALL + " | " + 
                  colors[self.grid[i+2]] + self.grid[i+2] + Style.RESET_ALL)
            if i < 6:
                print("---+---+---")
        print("\n")

    def display_reference(self):
        print(f"{Fore.YELLOW}Board positions:{Style.RESET_ALL}\n")
        reference = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
        for i in range(0, 9, 3):
            print(" " + reference[i] + " | " + reference[i+1] + " | " + reference[i+2])
            if i < 6:
                print("---+---+---")
        print("\n")

    def is_full(self):
        return " " not in self.grid

    def make_move(self, position, symbol):
        index = position - 1
        if index < 0 or index > 8:
            print(f"{Fore.RED}Invalid position. Choose a number between 1 and 9.{Style.RESET_ALL}")
            return False
        if self.grid[index] != " ":
            print(f"{Fore.RED}That spot is already taken. Try again.{Style.RESET_ALL}")
            return False
        self.grid[index] = symbol
        self.move_history.append((position, symbol))
        return True

    def undo_last_move(self):
        if self.move_history:
            position, _ = self.move_history.pop()
            self.grid[position - 1] = " "
            return True
        return False

    def check_winner(self, symbol):
        win_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
            [0, 4, 8], [2, 4, 6]              # Diagonals
        ]
        return any(all(self.grid[i] == symbol for i in combo) for combo in win_combinations)

# Test Board methods
b = Board()
print("Display reference:")
b.display_reference()
print("Display empty board:")
b.display()

## TicTacToe — game controller class

This class coordinates `Board` and `Player` objects. The implementation below is copied from the source notebook and demonstrates methods that control game flow.

In [ ]:
import random
from colorama import Fore, Style, init
init(autoreset=True)  # Initialize colorama

class TicTacToe:
    def __init__(self, player1, player2):
        self.board = Board()
        self.players = [player1, player2]
        self.current_player = player1
        self.game_history = []
        self.moves_count = 0
    
    def switch_player(self):
        self.current_player = self.players[1] if self.current_player == self.players[0] else self.players[0]
        if not self.current_player.is_ai:
            print(f"{Fore.CYAN}Now it's {self.current_player.name}'s turn{Style.RESET_ALL}")
    
    def play_turn(self):
        self.moves_count += 1
        if self.current_player.is_ai:
            position = self.current_player.get_move(self.board)
            print(f"{Fore.YELLOW}AI {self.current_player.name} chooses position {position}{Style.RESET_ALL}")
        else:
            try:
                position = int(input(f"{self.current_player.name}, enter position (1-9) or 0 to undo: "))
                if position == 0:
                    if self.board.undo_last_move():
                        self.switch_player()
                        self.moves_count -= 2
                        print(f"{Fore.YELLOW}Move undone!{Style.RESET_ALL}")
                    return True
            except ValueError:
                print(f"{Fore.RED}Please enter a number!{Style.RESET_ALL}")
                return True
        
        if self.board.make_move(position, self.current_player.symbol):
            self.board.display()
            
            if self.board.check_winner(self.current_player.symbol):
                print(f"{Fore.GREEN}🎉 {self.current_player.name} WINS in {self.moves_count} moves!{Style.RESET_ALL}")
                self.current_player.add_win()
                self.game_history.append({
                    'winner': self.current_player.name,
                    'moves': self.moves_count
                })
                return False
            
            if self.board.is_full():
                print(f"{Fore.YELLOW}It's a tie!{Style.RESET_ALL}")
                self.game_history.append({
                    'winner': 'Tie',
                    'moves': self.moves_count
                })
                return False
            
            self.switch_player()
            return True
        return True

    def display_stats(self):
        print(f"\n{Fore.CYAN}=== Game Statistics ==={Style.RESET_ALL}")
        for player in self.players:
            print(f"{player.name}: {player.wins} wins")
        print(f"Total games played: {len(self.game_history)}")
        if self.game_history:
            avg_moves = sum(game['moves'] for game in self.game_history) / len(self.game_history)
            print(f"Average moves per game: {avg_moves:.1f}")

# Quick demo to ensure class definitions are available
player1 = Player("Human", "X")
player2 = Player("AI", "O", is_ai=True)
game = TicTacToe(player1, player2)
print("Created TicTacToe game with players:")
print(game.players[0].name, game.players[0].symbol)
print(game.players[1].name, game.players[1].symbol, game.players[1].is_ai)

## AIPlayer subclass (optional)

The original notebook includes an `AIPlayer` subclass; include it here for completeness—it's an extension of `Player` that provides smarter `get_move` logic.

In [ ]:
class AIPlayer(Player):
    def __init__(self, name, symbol):
        super().__init__(name, symbol)
        self.is_ai = True
    
    def get_move(self, board):
        """
        Implements the AI strategy in this order:
        1. Win if possible
        2. Block opponent's winning move
        3. Take center
        4. Take corner
        5. Take any available space
        """
        # Try to win
        winning_move = self._find_winning_move(board, self.symbol)
        if winning_move is not None:
            return winning_move
        
        # Block opponent's winning move
        opponent_symbol = "O" if self.symbol == "X" else "X"
        blocking_move = self._find_winning_move(board, opponent_symbol)
        if blocking_move is not None:
            return blocking_move
        
        # Take center if available
        if board.grid[4] == " ":
            return 5
        
        # Take a corner if available
        corners = [0, 2, 6, 8]
        import random
        random.shuffle(corners)  # Add randomness to corner selection
        for corner in corners:
            if board.grid[corner] == " ":
                return corner + 1
        
        # Take any available space
        for i in range(9):
            if board.grid[i] == " ":
                return i + 1
    
    def _find_winning_move(self, board, symbol):
        """Check all possible moves to find a winning move"""
        for i in range(9):
            if board.grid[i] == " ":
                # Try move
                board.grid[i] = symbol
                if board.check_winner(symbol):
                    board.grid[i] = " "  # Reset the move
                    return i + 1
                board.grid[i] = " "  # Reset the move
        return None

## Simulated game demo

Use the code below to run a short simulated game or to drive the full interactive loop if you have a terminal. This demo is taken from the notebook and shows how classes collaborate.

In [ ]:
# Simulate a quick game to see all classes working together
print("=== SIMULATED GAME DEMO ===")
player1 = Player("Alice", "X")
player2 = Player("Bob", "O")
game = TicTacToe(player1, player2)

# Simulate some moves
moves = [
    (1, "Alice plays position 1"),
    (5, "Bob plays position 5"), 
    (2, "Alice plays position 2"),
    (6, "Bob plays position 6"),
    (3, "Alice plays position 3 - should win!")
]

print(f"Starting game: {player1.name} vs {player2.name}")
game.board.display_reference()
game.board.display()

for position, description in moves:
    print(f"\n--- {description} ---")
    
    # Make the move
    success = game.board.make_move(position, game.current_player.symbol)
    
    if success:
        game.board.display()
        
        # Check for winner
        if game.board.check_winner(game.current_player.symbol):
            print(f"🎉 {game.current_player.name} ({game.current_player.symbol}) WINS!")
            break
            
        # Check for tie
        if game.board.is_full():
            print("It's a tie!")
            break
            
        # Switch players
        game.switch_player()
    else:
        print("Move failed, trying next move...")

## Exercises

1. Add a `reset` method to `Board` that clears the grid and move history.
2. Add a `__str__` to `Board` that returns a single-line string representation of the grid.
3. Create a `HumanPlayer` subclass that overrides `get_move` to prompt the user and validate input.
4. Add difficulty levels to `AIPlayer` by changing its move heuristics.